In [1]:
import sympy
from symcircuit.symbolic_solver.domains import ExperimentResult

# Loading the Results of VLSI CMMF Exploration

In [2]:
results = ExperimentResult("VLSI_CMMF_POST_EXPERIMENT")
dir_of_results = results.find_results_file("Runs/VLSI_CMMF_Automated_NA")
results = results.load(f"{dir_of_results[0]}/results.pkl")

In [3]:
results.classifications_dict.keys()

dict_keys(['Z1_Z2_Z3_Z4_Z5_Z6', 'Z2_Z3_Z4_Z5_Z6', 'Z1_Z3_Z4_Z5_Z6', 'Z1_Z2_Z3_Z5_Z6', 'Z3_Z4_Z5_Z6', 'Z2_Z3_Z5_Z6', 'Z1_Z3_Z5_Z6', 'Z3_Z5_Z6'])

In [4]:
classifications_of_intrest = results.classifications_dict["Z1_Z2_Z3_Z4_Z5_Z6"]
len(classifications_of_intrest), classifications_of_intrest[0]

(4096,
 FilterClassification(valid=False, fType=X-INVALID-ORDER, parameters=None)zCombo=(R_1, R_2, R_3, R_4, R_5, R_6), transferFunc=R_1*R_2*R_4*R_6/(R_1*R_4*R_5 - R_2*R_3*R_4 + R_2*R_3*R_5 + R_2*R_4*R_5 + R_3*R_4*R_5), )

In [5]:
# df = results.flatten_classifications() # This way we can access all the classsifcation into a pandas df

# Examining the Outstanding Filters 

In [6]:
classifications_of_intrest = [classification 
                              for classification in classifications_of_intrest 
                              if (not classification.valid) or (not classification.fType in ["BP", "BS", "LP", "HP", "GE"] )]
fTypes = set(classification.fType for classification in classifications_of_intrest)
len(classifications_of_intrest), fTypes

(3828, {'X-INVALID-NUMER', 'X-INVALID-ORDER', 'X-INVALID-WZ'})

In [7]:
classifications_of_intrest[0:10]

[FilterClassification(valid=False, fType=X-INVALID-ORDER, parameters=None)zCombo=(R_1, R_2, R_3, R_4, R_5, R_6), transferFunc=R_1*R_2*R_4*R_6/(R_1*R_4*R_5 - R_2*R_3*R_4 + R_2*R_3*R_5 + R_2*R_4*R_5 + R_3*R_4*R_5), ,
 FilterClassification(valid=False, fType=X-INVALID-ORDER, parameters=None)zCombo=(R_1, R_2, R_3, R_4, R_5, 1/(C_6*s)), transferFunc=R_1*R_2*R_4/(s*(C_6*R_1*R_4*R_5 - C_6*R_2*R_3*R_4 + C_6*R_2*R_3*R_5 + C_6*R_2*R_4*R_5 + C_6*R_3*R_4*R_5)), ,
 FilterClassification(valid=False, fType=X-INVALID-ORDER, parameters=None)zCombo=(R_1, R_2, R_3, R_4, R_5, R_6 + 1/(C_6*s)), transferFunc=(C_6*R_1*R_2*R_4*R_6*s + R_1*R_2*R_4)/(s*(C_6*R_1*R_4*R_5 - C_6*R_2*R_3*R_4 + C_6*R_2*R_3*R_5 + C_6*R_2*R_4*R_5 + C_6*R_3*R_4*R_5)), ,
 FilterClassification(valid=False, fType=X-INVALID-ORDER, parameters=None)zCombo=(R_1, R_2, R_3, R_4, R_5, R_6/(C_6*R_6*s + 1)), transferFunc=R_1*R_2*R_4*R_6/(R_1*R_4*R_5 - R_2*R_3*R_4 + R_2*R_3*R_5 + R_2*R_4*R_5 + R_3*R_4*R_5 + s*(C_6*R_1*R_4*R_5*R_6 - C_6*R_2*R_3*R_4*R

In [8]:
single_classification = classifications_of_intrest[0]
# Since the original code classified the transfer functions by whether they fit to into a BiQuad filter, we classify them based on a different spec
single_classification.filterOrder, single_classification.fType, single_classification.parameters,  single_classification.zCombo, single_classification.valid

('BiQuad', 'X-INVALID-ORDER', None, (R_1, R_2, R_3, R_4, R_5, R_6), False)

In [9]:
count = {
    "X-CONST" : 0
}
s = sympy.symbols("s")

for i, classification in enumerate(classifications_of_intrest):
    tf = classification.transferFunc

    if not (s in tf.free_symbols): # TF id NOT a function of S - Pure amplifier/level-shifter
        ftype = "X-CONST"
        count[ftype] += 1
        classification.fType = "X-CONST"
        classification.filterOrder = "Zero"

    if classification.fType == "X-INVALID-WZ":
        # print(i, classification)
        pass

count

{'X-CONST': 2}

# Load the old classification into a classifier object

In [10]:
from symcircuit.symbolic_solver.filter import Filter_Classifier

classifier = Filter_Classifier()
classifier.overwrite_classifications(classifications_of_intrest)
classifier.isClassified()


2025-01-19 22:39:06,870 - symcircuit.symbolic_solver.filter - !!!! Clearning the filter classifications !!!!


True

In [11]:
# classifier.classifications

In [12]:
# classifier.classifyFilter(filterOrder="FirstOrder")

In [13]:
# classifier.summarizeFilterType()

In [17]:
classifications, count = classifier.findFilterInClassification(filterType="X-INVALID-ORDER", denom_order=4)
len(classifications)

X-INVALID-ORDER : 892


892

In [18]:
k, numer, denom = classifier.decompose_tf(classifications[0].transferFunc)


In [19]:
k.simplify()

C_3*C_5*R_1*R_2*R_6

In [20]:
numer

[Poly(s**2, s, domain='ZZ'), Poly(s + 1/(C_4*R_4), s, domain='ZZ(C_4,R_4)')]

In [21]:
roots = sympy.roots(denom[1])
unique_roots = [x for x in roots]
unique_roots

[-(-3*(C_3*R_1 + C_3*R_2 + C_4*R_2 + C_4*R_4 - C_5*R_2 + C_5*R_5)/(C_3*C_4*C_5*R_1*R_4*R_5 + C_3*C_4*C_5*R_2*R_4*R_5) + (C_3*C_4*R_1*R_4 + C_3*C_4*R_2*R_4 + C_3*C_5*R_1*R_5 + C_3*C_5*R_2*R_5 - C_4*C_5*R_2*R_4 + C_4*C_5*R_2*R_5 + C_4*C_5*R_4*R_5)**2/(C_3*C_4*C_5*R_1*R_4*R_5 + C_3*C_4*C_5*R_2*R_4*R_5)**2)/(3*(sqrt(-4*(-3*(C_3*R_1 + C_3*R_2 + C_4*R_2 + C_4*R_4 - C_5*R_2 + C_5*R_5)/(C_3*C_4*C_5*R_1*R_4*R_5 + C_3*C_4*C_5*R_2*R_4*R_5) + (C_3*C_4*R_1*R_4 + C_3*C_4*R_2*R_4 + C_3*C_5*R_1*R_5 + C_3*C_5*R_2*R_5 - C_4*C_5*R_2*R_4 + C_4*C_5*R_2*R_5 + C_4*C_5*R_4*R_5)**2/(C_3*C_4*C_5*R_1*R_4*R_5 + C_3*C_4*C_5*R_2*R_4*R_5)**2)**3 + (27/(C_3*C_4*C_5*R_1*R_4*R_5 + C_3*C_4*C_5*R_2*R_4*R_5) - 9*(C_3*R_1 + C_3*R_2 + C_4*R_2 + C_4*R_4 - C_5*R_2 + C_5*R_5)*(C_3*C_4*R_1*R_4 + C_3*C_4*R_2*R_4 + C_3*C_5*R_1*R_5 + C_3*C_5*R_2*R_5 - C_4*C_5*R_2*R_4 + C_4*C_5*R_2*R_5 + C_4*C_5*R_4*R_5)/(C_3*C_4*C_5*R_1*R_4*R_5 + C_3*C_4*C_5*R_2*R_4*R_5)**2 + 2*(C_3*C_4*R_1*R_4 + C_3*C_4*R_2*R_4 + C_3*C_5*R_1*R_5 + C_3*C_5*R_2*R_5

In [ ]:
(unique_roots[0] + unique_roots[1] + unique_roots[2]).simplify()

In [22]:
denom[1].as_expr().simplify()

(C_3*C_4*C_5*R_4*R_5*s**3*(R_1 + R_2) + s**2*(C_3*C_4*R_1*R_4 + C_3*C_4*R_2*R_4 + C_3*C_5*R_1*R_5 + C_3*C_5*R_2*R_5 - C_4*C_5*R_2*R_4 + C_4*C_5*R_2*R_5 + C_4*C_5*R_4*R_5) + s*(C_3*R_1 + C_3*R_2 + C_4*R_2 + C_4*R_4 - C_5*R_2 + C_5*R_5) + 1)/(C_3*C_4*C_5*R_4*R_5*(R_1 + R_2))

In [23]:
len(sympy.roots(sympy.Poly(denom[1], sympy.symbols("s"))))

3

In [24]:
(classifications[0].transferFunc)

(C_3*C_4*C_5*R_1*R_2*R_4*R_6*s**3 + C_3*C_5*R_1*R_2*R_6*s**2)/(s**4*(C_3*C_4*C_5*C_6*R_1*R_4*R_5*R_6 + C_3*C_4*C_5*C_6*R_2*R_4*R_5*R_6) + s**3*(C_3*C_4*C_5*R_1*R_4*R_5 + C_3*C_4*C_5*R_2*R_4*R_5 + C_3*C_4*C_6*R_1*R_4*R_6 + C_3*C_4*C_6*R_2*R_4*R_6 + C_3*C_5*C_6*R_1*R_5*R_6 + C_3*C_5*C_6*R_2*R_5*R_6 - C_4*C_5*C_6*R_2*R_4*R_6 + C_4*C_5*C_6*R_2*R_5*R_6 + C_4*C_5*C_6*R_4*R_5*R_6) + s**2*(C_3*C_4*R_1*R_4 + C_3*C_4*R_2*R_4 + C_3*C_5*R_1*R_5 + C_3*C_5*R_2*R_5 + C_3*C_6*R_1*R_6 + C_3*C_6*R_2*R_6 - C_4*C_5*R_2*R_4 + C_4*C_5*R_2*R_5 + C_4*C_5*R_4*R_5 + C_4*C_6*R_2*R_6 + C_4*C_6*R_4*R_6 - C_5*C_6*R_2*R_6 + C_5*C_6*R_5*R_6) + s*(C_3*R_1 + C_3*R_2 + C_4*R_2 + C_4*R_4 - C_5*R_2 + C_5*R_5 + C_6*R_6) + 1)

# Playground

In [ ]:
from sympy import symbols, Poly, factor, simplify, solveset, S

def decompose_tf(transfer_function, variable, assumptions={}):
    """
    Decomposes the denominator of a transfer function into factors, 
    grouping strictly by the specified variable (e.g., `s`).

    Args:
        transfer_function: A Sympy expression representing the transfer function.
        variable: The main variable to group by (e.g., `s`).
        assumptions: A dictionary of assumptions for symbolic variables.

    Returns:
        A dictionary where each key is a factor (grouped by the main variable),
        and the value contains its roots and properties.
    """
    # Apply assumptions to all symbols
    for var, assumption in assumptions.items():
        var._assumptions.update({assumption: True})
    
    # Extract the denominator
    denominator = transfer_function.as_numer_denom()[1]
    
    # Ensure the denominator is treated as a polynomial in the given variable
    poly = Poly(denominator, variable)
    
    # Factorize strictly with respect to `variable`
    factored = factor(poly.as_expr())
    
    # Inspect each factor
    factors = factored.as_ordered_factors()
    results = {}
    for factor_expr in factors:
        # Solve for the roots of this factor
        roots = solveset(factor_expr, variable, domain=S.Complexes)
        
        # Analyze each root (e.g., real part)
        roots_info = []
        for root in roots:
            real_part = simplify(root.as_real_imag()[0])
            roots_info.append({
                "root": root,
                "real_part": real_part
            })
        
        # Store results for this factor
        results[str(factor_expr)] = {
            "factor": factor_expr,
            "roots": roots_info,
            "degree": Poly(factor_expr, variable).degree()
        }
    
    return results

# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function
G = (s + R1) / (L1 * s**3 + (R2 + R3) * s**2 + 1 / C1 * s + 1 / C2)

# Decompose and inspect the factors
results = decompose_tf(G, s)
for factor, info in results.items():
    print(f"Factor: {info['factor']}")
    print(f"Degree: {info['degree']}")
    print("Roots:")
    for root_info in info["roots"]:
        print(f"  - Root: {root_info['root']}, Real part: {root_info['real_part']}")
    print()


In [ ]:
results.keys()

In [51]:
from sympy import symbols, Poly, factor, simplify, solveset, S

def decompose_tf(transfer_function):
    s = symbols("s")

    # Extract the numerator and denominator
    numerator, denominator = transfer_function.as_numer_denom()
 
    numerator = Poly(numerator, s)
    denominator = Poly(denominator, s)

    # Factorize strictly with respect to `variable`
    numerator = factor(numerator.as_expr())
    denominator = factor(denominator.as_expr())

    # Inspect each factor
    factors_numer = numerator.as_ordered_factors()
    factors_denom = denominator.as_ordered_factors()
    
    return factors_numer, factors_denom

# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function
G = (s + R1) / (((s+R3)*(s**2 + s*2*C2+ 2*C2**2)).expand())
G.expand()

# Decompose and inspect the factors
tup = decompose_tf(G)
numer = tup[0]
denom = tup[1]
